### 构建聊天机器人

#### 概述
我们将介绍如何设计和实现 LLM 驱动的聊天机器人的示例。 该聊天机器人将能够进行对话并记住以前的交互。

请注意，我们构建的这个聊天机器人将仅使用语言模型进行对话。 您可能正在寻找其他几个相关概念：

对话式 RAG：通过外部数据源实现聊天机器人体验

座席：构建一个可以执行操作的聊天机器人

本教程将介绍对这两个更高级的主题有帮助的基础知识，但如果您选择，请随意直接跳到那里。

#### 概念
以下是我们将使用的一些高级组件：

1. `聊天模型`。聊天机器人界面基于消息而不是原始文本，因此最适合聊天模型而不是文本 LLM。
2. `提示模板`，简化了组合默认消息、用户输入、聊天历史记录和（可选）其他检索上下文的提示的过程。
3. `聊天记录`，它允许聊天机器人“记住”过去的互动，并在回答后续问题时将其考虑在内。
4. 使用 `LangSmith` 调试和跟踪应用程序

我们将介绍如何将上述组件组合在一起以创建一个强大的对话式聊天机器人。

#### 设置
Jupyter 笔记本
本指南（以及文档中的大多数其他指南）使用 Jupyter 笔记本，并假定读者也使用 Jupyter 笔记本。

Jupyter 笔记本非常适合学习如何使用 LLM 系统，因为经常会出错（意外输出、API 关闭等），在交互式环境中浏览指南是更好地理解它们的好方法。

本教程和其他教程可能在 Jupyter 笔记本中运行最方便。有关如何安装的说明，请参阅此处。

#### 快速入门
首先，让我们学习如何单独使用语言模型。LangChain支持许多不同的语言模型，您可以互换使用 - 在下面选择您要使用的语言模型！

In [ ]:
pip install -qU langchain-openai

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")  # 这次，我们用阿里的，搞一波

 让我们首先直接使用模型。是LangChain`Runnables`的实例，这意味着它们公开了一个标准接口来与它们进行交互。只需简单地调用模型，我们就可以将消息列表传递给该方法。`ChatModel.invoke`

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好，我是法外狂徒张三")])

AIMessage(content='你好，不过你似乎记错了，你并不是"法外狂徒张三"。在现实生活中，我们都是独立的个体，有自己的名字和身份。如果你有任何问题或者需要帮助，可以告诉我，我会尽力提供帮助。', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 17, 'total_tokens': 66}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9b2a0f54-5cf4-4325-813a-ac920017db87-0')

现在使用的模型是没有记忆状态的，你们刚刚交流了一次了，但是如果你再去问他，他就失忆了

In [3]:
model.invoke([HumanMessage(content="我是谁?")])

AIMessage(content='您是您自己，没有人能代替您。您是独一无二的，无论您的名字、身份、性格、经历或思想都是属于您自己的。在人生的舞台上，您扮演着主角，塑造自己的生活和故事。如果您有任何更具体的问题或需要帮助的地方，请告诉我，我会尽力协助您。', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 11, 'total_tokens': 74}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4d2c3992-b0e9-45bc-a927-918dd3db3381-0')

让我们看一下 `LangSmith` 跟踪示例

我们可以看到，它没有将之前的对话变成上下文，也无法回答问题。 这带来了糟糕的聊天机器人体验！

为了解决这个问题，我们需要将整个对话历史记录传递到模型中。让我们看看这样做时会发生什么：

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好，我是王小美"),
        AIMessage(content="王小美你好，我能帮助你什么？"),
        HumanMessage(content="我是谁？"),
    ]
)

AIMessage(content='你是王小美。如果你有任何问题或需要帮助，请随时告诉我。', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 36, 'total_tokens': 51}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-78e87ef4-0317-4d8e-aa5b-9badcb864ac3-0')

现在我们可以看到我们得到了很好的回应！

这是支撑聊天机器人对话能力的基本思想。 那么，我们如何最好地实现这一点呢？

### 消息历史记录
我们可以使用 Message History 类来包装我们的模型并使其有状态。 这将跟踪模型的输入和输出，并将它们存储在某个数据存储中。 

然后，未来的交互将加载这些消息，并将它们作为输入的一部分传递到链中。 让我们看看如何使用它！

首先，让我们确保安装 ，因为我们将使用其中的集成来存储消息历史记录。`langchain-community`

In [ ]:
# ! pip install langchain_community

之后，我们可以导入相关类并设置我们的链，该链包装模型并添加此消息历史记录。

这里的一个关键部分是我们传入的函数。此函数应接受并返回 Message History 对象。

这用于区分单独的对话，并且在调用新链时应作为配置的一部分传入（我们将展示如何做到这一点。`get_session_historysession_id`, `session_id`

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

我们现在需要创建一个每次都传递到可运行对象中。此配置包含的信息不是直接输入的一部分，但仍然有用。

在本例中，我们希望包含一个 .这应该如下所示：`config`, `session_id`

In [7]:
config = {"configurable": {"session_id": "abc2"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="你好，我是王小美！")],
    config=config,
)

response.content

Parent run 258bf2c3-6cd4-4ff4-9540-4bca9597bbfd not found for run e2d2916f-a2a2-4b7e-8c22-b1723cfe5149. Treating as a root run.


'你好，王小美！很高兴认识你。有什么我能为你做的吗？'

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="我是谁？")],
    config=config,
)

response.content

Parent run c9f61ad6-3111-4f46-b165-97cb70a9d67c not found for run 02c4b735-31cf-464a-b493-cd98e9c8ecc3. Treating as a root run.


'你是王小美，我们刚才已经打过招呼了。'

很棒！我们的聊天机器人现在记住了关于我们的事情。如果我们把session id改为其他的 ，我们可以看到它重新开始新的对话。`session_id`

In [10]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="我是谁？")],
    config=config,
)

response.content

Parent run b69ef775-8aee-42f6-8d42-c86245d00ae7 not found for run f1fb4216-78d1-4d87-9c74-69faa6520760. Treating as a root run.


'您是您自己，没有人能代替您。您是独一无二的，无论您是谁都只能由您自己确定。如果您在寻求更深层次的哲学答案，那么您可能是您的思想、情感、经历和身份的总和，这是一个不断发展和变化的概念。在不同的上下文中，您可能是儿子/女儿、父亲/母亲、朋友、同事、爱人等等。总的来说，您是您自己的主体，存在于这个世界上的一个独立个体。'

但是，我们总是可以回到原始对话（因为我们将其保存在数据库中）

In [11]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="我是谁？")],
    config=config,
)

response.content

Parent run d81d9ae7-5577-4752-8a84-1a21767d7064 not found for run 7c0f6863-61fd-445e-b6f2-cdb334e8277c. Treating as a root run.


'你是王小美，我们似乎正在经历一次有趣的对话。如果我记错了，那请纠正我。'

这就是我们如何支持聊天机器人与许多用户进行对话的方式！

现在，我们所做的只是在模型周围添加一个简单的持久层。

我们可以通过添加提示模板开始使内容更加复杂和个性化。

### 提示词模板

提示模板有助于将原始用户信息转换为 LLM 可以使用的格式。在这种情况下，原始用户输入只是一条消息，我们将它传递给 LLM。现在让我们让它更复杂一点。

首先，让我们添加一个带有一些自定义指令的系统消息（但仍然将消息作为输入）。接下来，除了消息之外，我们还将添加更多输入。

首先，让我们添加一条系统消息。

为此，我们将创建一个 `ChatPromptTemplate`。我们将利用所有消息传递。`MessagesPlaceholder`

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个乐于助人的助手。尽你所能回答所有问题.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

请注意，这略微改变了输入类型 - 我们现在不是传入消息列表，而是传入一个带有键的字典，其中包含消息列表。`messages`

In [13]:
response = chain.invoke({"messages": [HumanMessage(content="你好，我是王小美")]})

response.content

'你好，王小美！很高兴能为你提供帮助。有什么可以帮到你的吗？'

现在，我们可以将其包装在与以前相同的 Messages History 对象中

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [15]:
config = {"configurable": {"session_id": "abc5"}}

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="哈，我是马小芳！")],
    config=config,
)

response.content

Parent run 2615a0e8-fb70-4ab7-bad3-9212e4b05779 not found for run c0378c09-b7d2-40b4-ab52-0d94ca999985. Treating as a root run.


'你好，马小芳！很高兴见到你。有什么我可以帮助你的吗？'

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="我是谁?")],
    config=config,
)

response.content

Parent run 6b468b62-9d60-4fbf-84f9-60c9482991c6 not found for run c9bdada8-8e94-44f8-813a-492f25ace2b9. Treating as a root run.


'在我们的对话中，你是马小芳，一个虚构或真实的人物。如果你有具体的问题或者需要讨论的话题，欢迎告诉我！'

棒！现在让我们的提示稍微复杂一点。让我们假设提示模板现在看起来像这样：

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个乐于助人的助手。尽你所能回答所有的问题 {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

请注意，我们已向提示添加了新输入。现在，我们可以调用链并传递我们选择的语言，比如说 西班牙语。`language`

In [19]:
response = chain.invoke(
    {"messages": [HumanMessage(content="你好，我是王小明")], "language": "Spanish"}
)

response.content

'¡Hola, Wang Xiaoming! ¿En qué puedo ayudarte hoy?'

现在，让我们将这个更复杂的链包装在 Message History 类中。这一次，由于输入中有多个键，我们需要指定用于保存聊天记录的正确键。

In [20]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [21]:
config = {"configurable": {"session_id": "abc11"}}

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="你好，我是范闲")], "language": "Spanish"},
    config=config,
)

response.content

Parent run f4c30955-cf0b-4fe2-9b92-80403cca053a not found for run 47a9e490-b257-46b0-804a-f499187ea0dc. Treating as a root run.


'¡Hola! ¿En qué puedo ayudarte hoy, Fan Xian?'

为了帮助您了解内部发生了什么，你可以去看看 `LangSmith` 跟踪

### 管理对话历史记录

构建聊天机器人时要了解的一个重要概念是如何管理对话历史记录。如果不进行管理，消息列表将无限制增长，并可能溢出 LLM 的上下文窗口。

因此，添加一个步骤来限制您传入的消息的大小非常重要。

重要的是，您需要在提示模板之前执行此操作，但在从消息历史记录加载以前的消息之后执行此操作。

为此，我们可以在提示符前面添加一个简单的步骤来适当地修改密钥，然后将该新链包装在 Message History 类中。

首先，让我们定义一个函数来修改传入的消息。让我们让它选择最新的消息。然后，我们可以通过在开始时添加它来创建一个新链。`messages` `k`

In [23]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

现在让我们试试吧！如果我们创建一个超过 10 条消息的消息列表，我们可以看到它不再记住早期消息中的信息。

In [24]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [25]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't know, what is your name?"

但是，如果我们询问最后十条消息中的信息，它仍然会记住它

In [29]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'You mentioned earlier that your favorite ice cream is vanilla.'

现在让我们将其包装在消息历史记录中

In [30]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [31]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="我是谁?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run b710b6d9-81a9-4410-875e-290b1bc264fe not found for run f7a62c38-32ea-4c07-ab43-15921790e8d8. Treating as a root run.


'您是您自己，没有人能代替您。您是独一无二的。'

聊天记录中现在有两条新消息。这意味着过去在我们的对话历史记录中可以访问的更多信息不再可用！

In [32]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 42f36382-3a3d-43a0-982e-54f936ed126f not found for run 86ec4930-1ad4-43db-945f-d230a5c458bf. Treating as a root run.


"I don't know, only you can know your own favorite ice cream flavor!"

如果你看一下 LangSmith，你可以看到 LangSmith 跟踪链路发生了什么！

### stream 流

现在我们有了一个功能聊天机器人。然而，聊天机器人应用程序的一个非常重要的用户体验考虑因素是流媒体。LLM 有时可能需要一段时间才能做出响应，因此为了改善用户体验，大多数应用程序所做的一件事就是在生成每个令牌时流回每个令牌。这允许用户查看进度。

这其实非常容易做到！

所有链都公开一个方法，使用消息历史记录的链也不例外。我们可以简单地使用该方法来返回流式响应。`stream`

In [33]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="你好，我是王晓梅，告诉我一个笑话！")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run 2768f021-b363-4563-87ee-c07de5b9dabe not found for run 3466c440-1527-457a-adcd-d58913554f6c. Treating as a root run.


|你好|，|王晓|梅|！这里有一个笑话|给你：为什么数学|书总是最不|开心的？因为它|总是到处都是问题|！希望你喜欢。||

完美

下一章，我们将了解RAG相关知识！